In [1]:
import torch
import torch.nn as nn
import torch.utils.data as torch_data
import torch.optim as optim
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!git clone https://github.com/nvanva/filimdb_evaluation.git
!cd filimdb_evaluation; bash ./init.sh; cd ..

Cloning into 'filimdb_evaluation'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 503 (delta 71), reused 85 (delta 40), pack-reused 377
Receiving objects: 100% (503/503), 119.53 MiB | 53.21 MiB/s, done.
Resolving deltas: 100% (263/263), done.
train.tsv
dev.tsv
train_small.tsv
dev_small.tsv
test.tsv
FILIMDB/
FILIMDB/train.labels
FILIMDB/train_unlabeled.texts
FILIMDB/test.texts
FILIMDB/train.texts
FILIMDB/dev.labels
FILIMDB/dev.texts
FILIMDB/dev-b.labels
FILIMDB/dev-b.texts
FILIMDB/test-b.texts
PTB/ptb.train.txt
PTB/
PTB/README
PTB/ptb.test.txt
PTB/ptb.valid.txt


In [3]:
from filimdb_evaluation.score import load_dataset_fast
datasets = load_dataset_fast(data_dir='./filimdb_evaluation/FILIMDB/')

Loading train set 
pos 7520
neg 7480
Loading dev set 
pos 4980
neg 5020
Loading test set 
unlabeled 25000


In [4]:
def loadPart(datasets, part):
    texts = []
    labels = []
    for i in range(len(datasets[part][1])):
        tokens = [w.lower() for w in nltk.word_tokenize(datasets[part][1][i])]
        labels.append(1 if datasets[part][2][i]=='pos' else 0)
        texts.append(tokens)
    return texts, labels

In [5]:
texts_train, labels_train = loadPart(datasets, 'train')
texts_val, labels_val = loadPart(datasets, 'dev')

In [6]:
vocab = set()
for text in texts_train:
    vocab |= set(text)
len(vocab)

88616

In [7]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip ./glove.6B.zip
w2ids = {}
ids = 1
w2ids["<pad>"] = 0 # Special <pad> token to make all examples of equal length inside each batch.
vecs = [torch.zeros((1, 200), requires_grad=True)]
with open('glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        word = parts[0]
        if(word in vocab):
            vec = torch.tensor([float(num) for num in parts[1:]], requires_grad = True).view(1,-1)
            w2ids[word] = ids
            ids += 1
            vecs.append(vec)
vecs = torch.cat(vecs, dim=0)
vecs.shape

--2020-12-06 20:41:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-06 20:41:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-06 20:41:11--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

torch.Size([54913, 200])

In [11]:
# фиксируем гиперпараметры обучения нашей модели.
train_epoches = 10
learning_rate = 1e-3
batch_size = 20
gradient_accumulation_steps = 2
final_learning_rate = 1e-5 

## Препроцессинг данных

Удобно оформлять ввиде класса наследника torch.utils.data.Dataset, потому что в pytorch уже реализованы такие частые операции, как: перемешивание данных, разбиение на батчи, приведение к формату torch tensor. Причем есть возможность делать это параллельно. 

In [12]:
#Чтобы представить данные в нужном формате надо перегрузить следующие методы, 
#суть которых понятна из названия. 
class ImdbData(torch_data.Dataset):
    def __init__(self, X, y, w2ids, aug):
        #инициализируем базовый класс
        super(ImdbData, self).__init__()
        self.X = X
        self.y = y
        self.w2i = w2ids
        self.aug = aug
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        tokens = self.X[idx]
        
        if(self.aug):
            half = len(tokens)//2
            if(np.random.binomial(1, 0.5)==1):
                tokens = tokens[:half]
            else:
                tokens = tokens[half:]
                
        ids = [self.w2i[t] for t in tokens if t in self.w2i]
        return (ids, self.y[idx])

### Dataloader
-dataset (Dataset) – Датасет  

-batch_size (python:int, optional) – Размер батча  

-shuffle (bool, optional) – Перемешивать ли данные на каждой эпохе обучения.  

-sampler (Sampler, optional) – определяет стратегию сэмплирования данных(shuffle=False)  

-batch_sampler (Sampler, optional) – определяет стратегию сэмплирования батчей.  

-num_workers (python:int, optional) – количество процессов в которых будет происходить формирвание батчей.  

-collate_fn (callable, optional) – собирает список примеров в минибатчи.  

-pin_memory (bool, optional) – копирование тензоров в бласть памяти из которой дальнейшая загрузка на гпу будет быстрее.   

-drop_last (bool, optional) – выбрасывать ли последний батч в случае, если он меньше остальных  

-timeout (numeric, optional) – ограничение по времени на формирование батча    

Сформируем итераторы по батчам из наших данных. Укажем размер батча, нужно ли данные перемешивать и количество потоков, которые будут формировать батчи. Параллелизм может ускорить формирование батча.

In [13]:
# This function will make a batch (an input and a target tensor) from a list of examples.
# To make these tensors, we add <pad> tokens to each examples to make them all the same length.
def collate_fn(batch_list):
    max_len = max([len(sample[0]) for sample in batch_list])
    
    tokens_tensor = [sample[0]+[0]*(max_len-len(sample[0])) for sample in batch_list]
    tokens_tensor = torch.tensor(tokens_tensor, requires_grad=False, dtype = torch.int64)
    
    labels_tensor = [sample[1] for sample in batch_list]
    labels_tensor = torch.tensor(labels_tensor, requires_grad=False, dtype = torch.int64)
    
    return (tokens_tensor, labels_tensor)

In [14]:
train_dataset = ImdbData(texts_train, labels_train, w2ids, aug = True)
val_dataset = ImdbData(texts_val, labels_val, w2ids, aug = False)
train_dataloader = torch_data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn = collate_fn)
val_dataloader =  torch_data.DataLoader(val_dataset, batch_size= batch_size, shuffle=False, collate_fn = collate_fn)
#test_dataloader = torch_data.DataLoader(ImdbData(texts_test, labels_test, w2v), batch_size= batch_size, shuffle=False)

# Модель

In [15]:
def create_emb_layer(weights_matrix):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    return emb_layer

Определим нашу модель как полносвязную нейронную сеть с двумя скрытыми слоями с функцией активации ReLU и dropout регуляризацией. В качестве входного представления текста будем использовать среднее арифметическое GLOVe  Также будем применять для регуляризации word dropout - каждый токен выкидывается из входного текста с вероятностью word_dropout.

В общем случае для этого достаточно отнаследоваться от nn.Module и перегрузить следующие 2 метода.  
в \_\_init\_\_ вызываем конструкторы слоев с нужными гиперпараметрами.  
в forward применяем слои в нужном порядке. 

In [16]:
import torch.nn.functional as F
from torch.distributions.bernoulli import Bernoulli

class Net(nn.Module):
    def __init__(self, vecs, word_dropout):
        super(Net, self).__init__()
        self.emb_layer = create_emb_layer(vecs)
        self.bern_sampler = Bernoulli(torch.tensor([1-word_dropout]))
        
        self.linear1 = nn.Linear(in_features=200, out_features=200)
        self.linear2 = nn.Linear(in_features=200, out_features=300)
        self.linear3 = nn.Linear(in_features=300, out_features=2) 
        
        self.dropout1 = nn.Dropout(p=0.1, inplace=False)
        self.dropout2 = nn.Dropout(p=0.1, inplace=False)
        #self.matrix = nn.Parameter(torch.zeros((3,3), requires_grad=True))

    def forward(self, tokens):
        pad_mask = (tokens != 0).unsqueeze(2)
        mask = pad_mask
        
        if(self.training):
            word_dropout_mask = self.bern_sampler.sample(sample_shape=torch.Size([tokens.shape[0], tokens.shape[1]]))
            word_dropout_mask = word_dropout_mask.to(mask.device)
            mask = mask*word_dropout_mask
        
        masked_inp  = (self.emb_layer(tokens)*mask)
        x = torch.mean(masked_inp, axis = 1)
        
        x = self.linear1(x)
        x = self.dropout1(x)
        x = F.relu(x)
        
        x = self.linear2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        
        x = self.linear3(x)
        return x
model = Net(vecs, 0.2)

In [17]:
torch.nn.Module.__setattr__??

Если же ваши вычисления придерживаются исключительно линейной логики, то код можно упростить с использованием Sequential

In [15]:
#model = nn.Sequential(nn.Linear(in_features=200, out_features=200),
#                        nn.Dropout(p=0.1, inplace=False),
#                        nn.ReLU(),
#                        nn.Linear(in_features=200, out_features=2))

Определим можно ли проводить вычисления на GPU.  Если у вас несколько GPU, то в качестве device можно просто указать ее номер.

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Working on', device)
print('For faster training ensure that a cuda (GPU) device is used, not cpu!')

Working on cuda
For faster training ensure that a cuda (GPU) device is used, not cpu!


Вычисления происходят там, где расположены ваши тензоры. Этой командой мы перемещаем параметры модели на нужный девайс.

In [19]:
model.to(device)

Net(
  (emb_layer): Embedding(54913, 200)
  (linear1): Linear(in_features=200, out_features=200, bias=True)
  (linear2): Linear(in_features=200, out_features=300, bias=True)
  (linear3): Linear(in_features=300, out_features=2, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

Передаем параметры модели в оптимизатор. Как вы видели ранее, тензоры сохраняют в себе градиенты. Оптимизатор достает их и применяет в соответствие с алгоритмом оптимизации.

In [20]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Зададим убывание learning rate. При каждом вызове метода step в ExponentialLR scheduler происходит домножение learning rate на параметр gamma. Будем делать scheduler step в конце каждой эпохи. Мы хотим чтобы за train_epoches шагов learning rate уменьшился до final_learning_rate. Из простой математики следует формула для gamma

In [21]:
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = (final_learning_rate/learning_rate)**(1/train_epoches))

CrossEntropyLoss по сути последовательное применение сначала softmax к логитам, потом logloss. Мы могли бы применить эти операции и сами, но их композиция, а главное градиенты по их композиции могут вычисляться оптимальнее.

In [22]:
loss_function = torch.nn.CrossEntropyLoss(weight=None, reduction='mean')

логгирование

In [23]:
#from torch.utils.tensorboard import SummaryWriter
!pip install tensorboardX
from tensorboardX import SummaryWriter
writer = SummaryWriter('./logs/')

     |████████████████████████████████| 317kB 24.1MB/s 


# Обучение

In [24]:
#цикл обучения одной эпохи
step = 0
def trainOneEpoch(model, device, train_loader, 
                  loss_function, optimizer, scheduler, writer, 
                  gradient_accumulation_steps):
    global step
    # Есть слои, которые при обучении и при тестировании ведут себя по разному
    # Например таким слоем является dropout слой. Вызовом этого метода мы меняем поведение слоев на то,
    # которое должно быть при обучении.
    model.train()
    #для агрегации функции потерь по батчам.
    losses = []
    for i, (X, y) in enumerate(train_loader):
        # тензоры параметров модели уже находятся на нужном для вычисления устройстве. Переместим туда же данные.
        X, y = X.to(device), y.to(device)
        # выходы модели подадим в функцию потерь 
        output = model(X) #model.forward()
        loss = loss_function(output, y)
        # При вызове backward() вычисленный градиент по каждому тензору прибавляются к полю тензора, накапливающему градиент по этому тензору. 
        # Следующий командой вы обнуляете это поле.
        # Если вызывать обнуление не перед каждым шагом, то можно реализовать аккамулирование градиентов.
        # Это бывает полезно, если вы хотите использовать большой рамер батча, но на него не хватает памяти GPU. 
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        # вычисление градиентов, прибавление к переменным аккамуляторам. 
        loss.backward()
        
        #применяем вычисленные градиенты согласно нашему алгоритму оптимизации.
        if((i+1)%gradient_accumulation_steps==0):
            optimizer.step()
            optimizer.zero_grad()
        #этой комндой мы получем numpy представление тензора. И залогируем его
        writer.add_scalar("train loss", loss.item(), step)
        step += 1
    scheduler.step()

## forward vs __call__  
Управлять значениями параметров и градиентами вы можете через непосредственно тензоры и поле grad. Управлять входами и выходами слоев и градиентами по входам и выходам вы можете через hooks. Некоторые сложные слои могут использовать hooks в  своей реализации. \_\_call\_\_ заботится о hooks. Поэтому его вызов предпочтительнее.

In [25]:
model.__call__??

In [26]:
def evalModel(model, test_loader, batch_size, device, loss_function):
    model.eval()
    S = 0
    S_loss = 0
    num_samples = 0
    num_batches = 0
    for X,y in test_loader:
        X, y = X.to(device), y.to(device)
        output = model(X)
        S_loss += loss_function(output, y).item()
        S+=torch.sum(torch.argmax(output, axis=1) != y).item()
        num_samples += X.shape[0]
        num_batches += 1
    return (S/num_samples, S_loss/num_batches) 

In [27]:
from matplotlib import pyplot as plt
for epoch in range(train_epoches):
    trainOneEpoch(model, 
                 device, 
                 train_dataloader, 
                 loss_function, 
                 optimizer, 
                 scheduler, writer, gradient_accumulation_steps)
    
    
    val_acc, val_loss = evalModel(model, val_dataloader, 
                                                batch_size, device, loss_function)
    
    train_acc, train_loss = evalModel(model, train_dataloader, 
                                                batch_size, device, loss_function)
    
    writer.add_scalar("val accuracy", val_acc, epoch)
    
    writer.add_scalar("train accuracy", train_acc, epoch)
    
    writer.add_scalar("val loss", val_loss, epoch)
    
    print('Train/valid ERR: %.3f/%.3f Train/valid loss: %.5f/%.5f' % (train_acc, val_acc, train_loss, val_loss))



Train/valid ERR: 0.172/0.158 Train/valid loss: 0.39360/0.36844
Train/valid ERR: 0.131/0.125 Train/valid loss: 0.32166/0.31668
Train/valid ERR: 0.115/0.115 Train/valid loss: 0.28386/0.28676
Train/valid ERR: 0.108/0.113 Train/valid loss: 0.26672/0.27788
Train/valid ERR: 0.100/0.110 Train/valid loss: 0.24954/0.27044
Train/valid ERR: 0.096/0.109 Train/valid loss: 0.24228/0.26852
Train/valid ERR: 0.093/0.109 Train/valid loss: 0.23543/0.26703
Train/valid ERR: 0.093/0.108 Train/valid loss: 0.23690/0.26506
Train/valid ERR: 0.095/0.108 Train/valid loss: 0.23775/0.26456
Train/valid ERR: 0.087/0.107 Train/valid loss: 0.22624/0.26435


In [ ]:
#!tensorboard --logdir ./logs/

In [ ]:
#Померяем error rate на тесте
#evalModel(model, test_dataloader, batch_size, device)

In [ ]:
# Поговорим еще немного о тензорах

Variable - в старых версиях для этих объектов отслеживалась история изменений. 
Variable.data способ получить тензор лежащий внутри Variable.   
В более поздних версиях тензор смешали с Variable, но в библиотеке эти методы остались для совместимости.
Вместо .data лучше использовать detach()

### view vs reshape
Возврщает тензор с теми же данными, но другой формы. 

In [ ]:
A = torch.arange(9)
B = A.view((3,3))
B[0,0]=100
A

tensor([100,   1,   2,   3,   4,   5,   6,   7,   8])

reshape по возможности делает также, но может вернуть и копию. Поэтому в работе reshape есть некоторая неопределенность, которую надо учитывать.
Зато reshape умеет работать с non-contiguous тензорами.

contiguous tensor тензор который хранится в непрерывном участке памяти. В следствие транспонирований или взятия среза результирующий тензор вполне может утрать это свойтво.

In [ ]:
x=np.arange(12).reshape(3,4).copy()
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [ ]:
x.T.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False